In [ ]:
!hostname

In [ ]:
import torch
# import torchvision.models as models
import torch.distributed as dist
# import torch.optim as optim
import scipy.stats as stats
from pathlib import Path
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from rich.progress import track


# # oialr_checkpoint_folder = "/hkfs/work/workspace/scratch/qv2382-madonna/madonna/models/svd-final/12/weights/epoch_299.pt"
# oialr_checkpoint_folder = "/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/svd-nodroppath/2/weights/epoch_299.pt"
# # baseline_checkpoint_folder = "/hkfs/work/workspace/scratch/qv2382-madonna/madonna/models/baseline-final/8/weights/epoch_299.pt"
# baseline_checkpoint_folder = "/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/baseline-nodroppath/1/weights/epoch_299.pt"

checkpoints = {}
# ortho-sigma
# /hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/42/weights3
# /hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/42/weights2
# /hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/43/weights1
# /hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/44/weights0
checkpoints["ortho-sigma"] = {
    0: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/44/weights0"),
    1: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/43/weights1"),
    2: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/42/weights2"),
    3: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug/42/weights3"),
}

checkpoints["rand-sigma"] = {
    0: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug-randsigma/0/weights0"),
    1: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug-randsigma/1/weights1"),
    2: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug-randsigma/2/weights2"),
    3: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-debug-randsigma/0/weights3"),
}

checkpoints_resnet18 = {}
checkpoints_resnet18["ortho-sigma"] = {
    # 1 : Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma/13/weights1"),
    # 0: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma/13/weights0"),
    # 2: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma/13/weights2"),
    # 3: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma/13/weights3"),
    # 3: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma/9/weights3"),
    # 0: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma/10/weights0"),
    # 1: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma/11/weights1"),
    # 2: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma/12/weights2"),
    1: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma-2/1/weights1"),
    2: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma-2/2/weights2"),
    0: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma-2/3/weights0"),
    3: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-orthosigma-2/4/weights3"),
}

checkpoints_resnet18["rand-sigma"] = {
    3: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-randsigma/0/weights3"),
    0: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-randsigma/1/weights0"),
    2: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-randsigma/2/weights2"),
    1: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-randsigma/3/weights1"),
}
checkpoints_resnet18["random"] = {
    3: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-random/0/weights3"),
    0: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-random/0/weights0"),
    2: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-random/0/weights2"),
    1: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-random/0/weights1"),
}

checkpoints_resnet18["unified"] = {
    3: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-unified/0/weights3"),
    0: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-unified/1/weights0"),
    2: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-unified/1/weights2"),
    1: Path("/hkfs/work/workspace/scratch/qv2382-madonna-ddp/madonna/models/fed-unified/1/weights1"),
}

combos = [[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]]
epochs = ["000", "024", "049", "074", "099", "124", "149", "174", "199", "224", "249", "274", "299", ]  # 

# svd_model_dict = torch.load(oialr_checkpoint_folder, map_location="cpu")
# baseline_model_dict = torch.load(baseline_checkpoint_folder, map_location="cpu")
# print(list(baseline_model_dict["model"].keys()))

# get 2D reps for a checkpoint (fn)
def get_2d_reps(model_dict, use_qr, float64=False):
    ret_dict = {}
    # nbytes = 0
    for n in model_dict['model']:
        p = model_dict['model'][n]
        # print(n, p.shape, p.nbytes)
        if p.ndim < 2:
            continue

        hld = p
        shp = p.shape
        if p.ndim > 2:  # collapse down to 2D
            hld = p.view(p.shape[0], -1)
        trans = hld.shape[0] < hld.shape[1]
        if trans:  # make 2D rep TS
            hld = hld.T

        if float64:
            hld = hld.to(torch.float64)
        if not use_qr:
            u, s, vh = torch.linalg.svd(hld, full_matrices=False)
            ret_dict[n] = {'u': u, 's': s, 'vh': vh, 'trans': trans, "w": hld, "shp": shp}
        else:
            q, r = torch.linalg.qr(hld, mode="complete")
            ret_dict[n] = {'q': q, "r": r, 'trans': trans, "w": hld, "shp": shp}
        
    return ret_dict

def get_2d_reps_from_names(model_dict, base_names, append_names, use_qr, float64=False):
    ret_dict = {}
    # nbytes = 0
    for base in base_names:
        p = model_dict['model'][base]
        # print(base, p.shape, p.nbytes)
        if p.ndim < 2:
            continue

        hld = p
        shp = p.shape
        if p.ndim > 2:  # collapse down to 2D
            hld = p.view(p.shape[0], -1)
        trans = hld.shape[0] < hld.shape[1]
        if trans:  # make 2D rep TS
            hld = hld.T

        cat_dim = None

        for app in append_names[base]:
            app_p = model_dict['model'][app]
            if cat_dim is None:
                cat_dim = torch.nonzero(torch.tensor(hld.shape) == torch.tensor(app_p.shape)).flatten()[0].item()
                # since its the same dim, need to cat it on the OTHER dim...
                cat_dim = (cat_dim + 1) % 2
            hld = torch.cat([hld, app_p.unsqueeze(cat_dim)], dim=cat_dim)

        if float64:
            hld = hld.to(torch.float64)

        if not use_qr:
            u, s, vh = torch.linalg.svd(hld, full_matrices=False)
            ret_dict[base] = {'u': u, 's': s, 'vh': vh, 'trans': trans, "w": hld, "cat_dim": cat_dim, "append_names": append_names[base], "shp": shp}
        else:
            q, r = torch.linalg.qr(hld, mode="reduced")
            ret_dict[base] = {'q': q, "r": r, 'trans': trans, "w": hld, "cat_dim": cat_dim, "append_names": append_names[base], "shp": shp}
    
    return ret_dict

def get_1d_associated_weights(model_dict):
    wgts_to_join = {}
    last_miltidim_w = None
    ignore_list = []  # "pos_embed", "cls_token"
    ignore_endings = ("running_mean", "running_var")
    for n in model_dict["model"]:
        p = model_dict["model"][n]
        if n in ignore_list or n.endswith(ignore_endings):
            continue
        if p.ndim > 1:  # and last_miltidim_w is None:
            last_miltidim_w = n
            if n in wgts_to_join:
                raise ValueError
            wgts_to_join[n] = []
        elif last_miltidim_w is not None and p.ndim == 1:
            wgts_to_join[last_miltidim_w].append(n)
    return wgts_to_join

def merge_means_vars(inmodels: list, epoch: str):
    # TODO: this will need to be parallized in a better way....heat-style?
    # do a pairwise update of the means and vars in both 
    num_models = len(inmodels)
    if num_models == 1:
        raise ValueError("need multiple models for merging")
    # n0 = num_samples[epoch]
    for layer in inmodels[0]:  # should be iterating over the layers now (TODO: DOUBLE CHECK)
        # print(layer)
        if not layer.endswith("mean"):
            continue
        # only work on mean layers, get vars here as well (if they exist)
        # go back and get the var tensor as well to update the batch norms
        try: 
            var0 = inmodels[0][layer[:-4] + "var"]
        except KeyError:
            var0 = None
        mu0 = inmodels[0][layer]
        n0 = inmodels[0][layer[:-12] + "num_batches_tracked"]
        # print(f"{layer}: mu0: {mu0}")
        for ml in inmodels[1:]:
            mul = ml[layer]
            
            nl = ml[layer[:-12] + "num_batches_tracked"]
            n = nl + n0
            delta = mul - mu0
            mu = mu0 + nl * (delta / n)
            # print(f"{layer}: mu: {mu.mean():.4f} -> {mu0.mean():.4f} {mul.mean():.4f}")
            mu0.zero_()
            mu0.add_(mu)

            if var0 is not None:
                varl = ml[layer[:-4] + "var"]
                var_m = (var0 * (n0 - 1) + varl * (nl - 1) + (delta**2) * n0 * nl / n) / (n - 1)
                # print(f"{layer}: var: {var_m.mean():.4f} -> {var0.mean():.4f} {varl.mean():.4f}")
                var0.zero_()
                var0.add_(var_m)
            n0 = n
        inmodels[0][layer[:-12] + "num_batches_tracked"] = n0

In [ ]:
load_models = True
models = {}
map_location = "cuda:0"  #"gpu:0" if torch.cuda.is_available() else "cpu"
if load_models:
    for rank in range(4):
        models[rank] = {}
        for epoch in epochs:
            models[rank][epoch] = torch.load(checkpoints_resnet18["ortho-sigma"][rank] / f"epoch_{epoch}.pt", map_location=map_location)
            # models[rank][epoch] = torch.load(checkpoints["ortho-sigma"][rank] / f"epoch_{epoch}.pt", map_location=map_location)
        print(f"loaded models for rank: {rank}")

In [ ]:
def get_rotation_btw_bases(basis_to_rotate, target_basis):
    # TODO: rename basis_a -> to_rotate and basis_b -> target_basis
    # create a rotation to rotate basis_a to basis_b
    # returns: R so that R @ A == B

    rot = torch.zeros((basis_to_rotate.shape[0], basis_to_rotate.shape[0]), dtype=basis_to_rotate.dtype, device=basis_to_rotate.device)
    for col in range(basis_to_rotate.shape[1]):
        rot += torch.kron(basis_to_rotate[:, col], target_basis[:, col].unsqueeze(1))
    return rot

In [ ]:
# is the rotation method the same as the average?
r0combos = [1, 2, 3]
cat_1d = True
use_qr = False
which_epoch = epochs[-1]
float64 = False

from collections import OrderedDict

# del model_0, rep_dict_0, rep_dict_l
new_model_dict = OrderedDict()

# if not load_models:
#     model_0 = torch.load(checkpoints_resnet18["random"][0] / f"epoch_{which_epoch}.pt", map_location=map_location)
# else:
model_0 = models[0][which_epoch]

new_model_dict = model_0["model"].copy()

if cat_1d:
    names = get_1d_associated_weights(model_0)
    rep_dict_0 = get_2d_reps_from_names(model_dict=model_0, base_names=names.keys(), append_names=names, use_qr=use_qr, float64=float64)
else:
    rep_dict_0 = get_2d_reps(model_dict=model_0, use_qr=use_qr, float64=float64)

for layer in rep_dict_0:  # create average of weights
    rep_dict_0[layer]['w_avg'] = rep_dict_0[layer]['w'] / 4

# get full Us and Vs
for layer in rep_dict_0:
    layer_0 = rep_dict_0[layer]
    uf, _, vhf = torch.linalg.svd(layer_0['w'], full_matrices=True)
    layer_0["uf"] = uf
    layer_0["vhf"] = vhf

for partner in r0combos:  # combos are 2d tuples with rankA, rankB
    # getting results
    # if not load_models:
    #     model_l = torch.load(checkpoints["random"][partner] / f"epoch_{which_epoch}.pt", map_location=map_location)
    # else:
    model_l = models[partner][which_epoch]
        
    # get append lists:
    if cat_1d:
        rep_dict_l = get_2d_reps_from_names(model_dict=model_l, base_names=names.keys(), append_names=names, use_qr=use_qr, float64=float64)
    else:
        rep_dict_l = get_2d_reps(model_dict=model_l, use_qr=use_qr, float64=float64)

    # loop through the dict and move the 
    for layer in rep_dict_l:  # keys are the same
        # do average here
        rep_dict_0[layer]['w_avg'] += rep_dict_l[layer]['w'] / 4

        layer_0 = rep_dict_0[layer]
        layer_l = rep_dict_l[layer]
        ufl, _, vhf = torch.linalg.svd(layer_l['w'], full_matrices=True)

        u0 = layer_0['u']
        uf0 = layer_0['uf']
        ul = layer_l['u']
        
        v0 = layer_0['vh'].T
        vf0 = layer_0['vhf'].T
        vl = layer_l['vh'].T
        vfl = vhf.T
        ru = get_rotation_btw_bases(basis_to_rotate=u0.T, target_basis=ul.T)
        rv = get_rotation_btw_bases(basis_to_rotate=v0.T, target_basis=vl.T)

        sim = (u0 @ v0.T) @ (ul @ vl.T).T

        if layer_0['s'].ndim == 1:
            layer_0['s'] = layer_0['s'].diag()
        layer_0['s'] += ru.T @ layer_l['s'].diag() @ rv
        # layer_0['s'] += layer_l['s'].diag() @ sim
        # print(partner, layer, layer_l['s'][:10])  
        # layer_0['s'] += layer_l['s'].diag()
        # layer_0['w'] += (sim @ layer_l['w']) / 4

for layer in rep_dict_0:  # finish average of weights
    u0, vh0 = rep_dict_0[layer]['u'], rep_dict_0[layer]['vh']
    sig = rep_dict_0[layer]['s'] 

    w = (u0 @ sig @ vh0) / 4
    w_avg = rep_dict_0[layer]['w_avg']
    w = rep_dict_0[layer]['w']
    # ua, sa, vha = torch.linalg.svd(w_avg, full_matrices=False)
    # uw, sw, vhw = torch.linalg.svd(w, full_matrices=False)    
    # print('avg sig', sa[:10])
    # print('combi', sw[:10])

    rep_dict_0[layer]['w_avg'] = w_avg
    rep_dict_0[layer]['w'] = w
    print(f"w_avg: {layer}: {w_avg.mean():.5f} {w_avg.min():.5f} {w_avg.max():.5f} {w_avg.std():.5f} ")
    print(f"w: {layer}: {w.mean():.5f} {w.min():.5f} {w.max():.5f} {w.std():.5f} ")

merge_means_vars(inmodels=[new_model_dict, *[models[i][which_epoch]['model'] for i in range(1, 4)]], epoch=which_epoch)

In [ ]:
# undo 1D concat and add to new_model_dict
avg_dict = {}

for layer in list(rep_dict_0.keys()):
    layer_dict = rep_dict_0[layer]

    if "cat_dim" in layer_dict:
        # this is the time to undo the cat-ing
        sliced = 0
        sl_dim = layer_dict["cat_dim"]
        for cat1dlayer in reversed(layer_dict["append_names"]):  # have to work from outside in
            # loop through the appended layers in revered order (outside -> in)
            sliced += 1
            sl = [slice(None), ] * layer_dict['w'].ndim
            sl[sl_dim] = -sliced
            # rep_dict_0[cat1dlayer] = {}
            # rep_dict_0[cat1dlayer]['w'] = layer_dict['w'][sl].clone()
            old_weight = new_model_dict[cat1dlayer]

            weight = layer_dict['w'][sl].clone()
            new_model_dict[cat1dlayer] = weight

            avg_weight = layer_dict['w_avg'][sl].clone()
            avg_dict[cat1dlayer] = avg_weight
            
            print(f"old {cat1dlayer}, {old_weight.mean():.4f}, {old_weight.min():.4f}, {old_weight.max():.4f}, {old_weight.std():.4f}")
            print(f"avg {cat1dlayer}, {avg_weight.mean():.4f}, {avg_weight.min():.4f}, {avg_weight.max():.4f}, {avg_weight.std():.4f}")
            print(f"{cat1dlayer}, {weight.mean():.4f}, {weight.min():.4f}, {weight.max():.4f}, {weight.std():.4f}")
        # remove the last `sliced` layers from the weight matrix
        sl = [slice(None), ] * layer_dict['w'].ndim
        sl[sl_dim] = slice(-sliced)
        weight = layer_dict['w'][sl]
        avg_weight = layer_dict['w_avg'][sl]
    else:
        weight = layer_dict['w']
        avg_weight = layer_dict['w_avg']

    if layer_dict['trans']:
        weight = weight.T
        avg_weight = avg_weight.T

    if weight.shape != layer_dict['shp']:
        weight = weight.reshape(layer_dict['shp'])
        avg_weight = avg_weight.reshape(layer_dict['shp'])
    
    old_weight = new_model_dict[layer]
    avg_dict[layer] = avg_weight
    new_model_dict[layer] = avg_weight.clone()

    print(f"old {layer}, {old_weight.mean():.4f}, {old_weight.min():.4f}, {old_weight.max():.4f}, {old_weight.std():.4f}")
    print(f"avg {layer}, {avg_weight.mean():.4f}, {avg_weight.min():.4f}, {avg_weight.max():.4f}, {avg_weight.std():.4f}")
    print(f"{layer}, {weight.mean():.4f}, {weight.min():.4f}, {weight.max():.4f}, {weight.std():.4f}")
    # print(layer, layer_dict['w'].shape)
# for layer in rep_dict_0:
#     print(layer, rep_dict_0[layer]['w'].shape)

In [ ]:
# normal average without anything else
model_avg = models[0][which_epoch]["model"].copy()
for lay in model_avg:
    # print(lay)
    for rank in range(1, 4):
        model_avg[lay] += models[rank][which_epoch]["model"][lay]
    if model_avg[lay].dtype == torch.long:
        model_avg[lay] *= 4
    else:
        model_avg[lay] /= 4

    if lay in new_model_dict:
        print(f"{lay}: comp to avg: {torch.allclose(model_avg[lay], new_model_dict[lay])} -> {(model_avg[lay] - new_model_dict[lay]).max()}")


In [ ]:
# set model weights
import timm
from timm.data.transforms_factory import create_transform

model = timm.models.resnet18(num_classes=10)
model = model.to("cuda:0")
model = model.float()
with torch.no_grad():
    # model.load_state_dict(models[0]["299"]['model'])
    # model.load_state_dict(models[1]["299"]["model"])
    model.load_state_dict(new_model_dict)
    # model.load_state_dict(model_avg)
model = model.eval()

In [ ]:
from torchmetrics import MetricCollection, Precision, Recall
from torchmetrics.classification import MulticlassF1Score, MulticlassPrecision, MulticlassRecall

device = "cuda:0"
val_metrics = MetricCollection(
    [
        MulticlassF1Score(num_classes=10),
        MulticlassPrecision(num_classes=10),
        MulticlassRecall(num_classes=10),
    ],
).to(device)

In [ ]:
from enum import Enum
from torch.utils.data import Subset

class Summary(Enum):
    NONE = 0
    AVERAGE = 1
    SUM = 2
    COUNT = 3


class AverageMeter:
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=":f", summary_type=Summary.AVERAGE, pg=None):
        self.name = name
        self.fmt = fmt
        self.summary_type = summary_type
        self.reset()
        self.pg = pg

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def all_reduce(self):
        if torch.cuda.is_available():
            device = torch.device("cuda")
        elif torch.backends.mps.is_available():
            device = torch.device("mps")
        else:
            device = torch.device("cpu")
        total = torch.tensor([self.sum, self.count], dtype=torch.float32, device=device)
        dist.all_reduce(total, dist.ReduceOp.SUM, async_op=False, group=self.pg)
        self.sum, self.count = total.tolist()
        # self.avg = self.sum / self.count
        self.avg = total[0] / total[1]  # self.sum / self.count

    def __str__(self):
        fmtstr = "{name} {val" + self.fmt + "} ({avg" + self.fmt + "})"
        return fmtstr.format(**self.__dict__)

    def summary(self):
        # fmtstr = ""
        if self.summary_type is Summary.NONE:
            fmtstr = ""
        elif self.summary_type is Summary.AVERAGE:
            fmtstr = "{name} {avg:.3f}"
        elif self.summary_type is Summary.SUM:
            fmtstr = "{name} {sum:.3f}"
        elif self.summary_type is Summary.COUNT:
            fmtstr = "{name} {count:.3f}"
        else:
            raise ValueError("invalid summary type %r" % self.summary_type)

        return fmtstr.format(**self.__dict__)


class ProgressMeter:
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix
        self.rank = 0 if not dist.is_initialized() else dist.get_rank()

    def display(self, batch, log=None):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        # if self.rank == 0:
        #     # log.info("\t".join(entries))
        if log is None:
            print(" ".join(entries))
        else:
            log.info(" ".join(entries))

    def display_summary(self, log=None, printing_rank=None):
        entries = [" *"]
        entries += [meter.summary() for meter in self.meters]
        if printing_rank or printing_rank is None:
            if log is None:
                print(" ".join(entries))
            else:
                # console.print(" ".join(entries))
                log.info(" ".join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = "{:" + str(num_digits) + "d}"
        return "[" + fmt + "/" + fmt.format(num_batches) + "]"


def accuracy(output, target, topk=(1,), mixup=False):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        if not mixup:
            pred = pred.t()
            correct = pred.eq(target.view(1, -1).expand_as(pred))
            res = []
            for k in topk:
                correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
                res.append(correct_k.mul_(100.0 / batch_size))
            return res
        else:
            maxk = max(topk)
            batch_size = target.size(0)
            if target.ndim == 2:
                target = target.max(dim=1)[1]

            _, pred = output.topk(maxk, 1, True, True)
            pred = pred.t()
            correct = pred.eq(target[None])

            res = []
            for k in topk:
                correct_k = correct[:k].flatten().sum(dtype=torch.float32)
                res.append(correct_k * (100.0 / batch_size))
            return res


In [ ]:
import time

criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)

def run_validate(loader, base_progress=0):
    # rank = 0 if not dist.is_initialized() else dist.get_rank()
    with torch.no_grad():
        end = time.time()
        num_elem = len(loader) - 1
        for i, data in enumerate(loader):
            
            images = data[0]
            target = data[1]
            i = base_progress + i
            images = images.to(device, non_blocking=True)
            target = target.to(device, non_blocking=True)

            data_time.update(time.time() - end)

            if torch.any(torch.isnan(images)):
                # for n, p in model.named_parameters():
                # print(f"{n}: {p.mean():.4f}, {p.min():.4f}, {p.max():.4f}, {p.std():.4f}")
                raise ValueError("NaN in images... - VAL")

            # compute output
            output = model(images)
            loss = criterion(output, target)
            # argmax = torch.argmax(output.output, dim=1).to(torch.float32)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))
            val_metrics.update(output, target)

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            # print(output[0])
            if not torch.isfinite(loss):
                # for n, p in model.named_parameters():
                for n, p in model.named_parameters():
                    print(f"{n}: {p.mean():.4f}, {p.min():.4f}, {p.max():.4f}, {p.std():.4f}, {torch.any(torch.isfinite(p))}")
                raise ValueError("NaN loss - VAL")
            if (i % 50 == 0 or i == num_elem):
                argmax = torch.argmax(output, dim=1).to(torch.float32)
                print(
                    f"output mean: {argmax.mean().item()}, max: {argmax.max().item()}, ",
                    f"min: {argmax.min().item()}, std: {argmax.std().item()}",
                )
                progress.display(i + 1)
            # return

batch_time = AverageMeter("Time", ":6.3f", Summary.NONE)
data_time = AverageMeter("Data", ":6.3f", Summary.NONE)
losses = AverageMeter("Loss", ":.4f", Summary.AVERAGE)
top1 = AverageMeter("Acc@1", ":6.2f", Summary.AVERAGE)
top5 = AverageMeter("Acc@5", ":6.2f", Summary.AVERAGE)
progress = ProgressMeter(
    len(test_loader) + (len(test_loader.sampler) < len(test_loader.dataset)),
    [batch_time, losses, top1, top5],
    prefix="Test: ",
)

# switch to evaluate mode
model.eval()
vt1 = time.perf_counter()
run_validate(test_loader)

if len(test_loader.sampler) < len(test_loader.dataset):
    aux_val_dataset = Subset(
        test_loader.dataset,
        range(len(test_loader.sampler), len(test_loader.dataset)),
    )
    aux_val_loader = torch.utils.data.DataLoader(
        aux_val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=6,
        pin_memory=True,
    )
    run_validate(aux_val_loader, len(test_loader))
val_time_total = time.perf_counter() - vt1

progress.display_summary()

print(top1.avg, losses.avg)